In [1]:
import numpy as np
from datasets import load_dataset
import re

from Attention.attention_block import attention_block as ab
from Attention.attention_head import attention_head as ah

import base_transformer as bt

%load_ext autoreload
%autoreload 2

In [2]:
import feed_forward as ff
test_layer = ff.neuron_layer(100, 100, 'relu', 8, .0001, .001)

In [82]:

test_embedding = np.random.normal(0, 1, size=(9,512))
test_head = ah(512, 64)

# print(np.sum(test_head.q))
# test_head.calculate_q_k_v(test_embedding)

test_head.masked_attention_score(test_embedding)

# print(test_head.W_v.shape)

(9, 9)
(9, 64)


In [52]:
test_block = ab(num_heads=8, input_dimension=512, head_output_dimension=64, block_output_dimension=128)

# test_block.W_o.shape

test_block.multi_head_attention(test_embedding).shape

# test_block.heads['head_1'].W_k.shape

# test_block.attention_scores['head_0']

(9, 512)


(9, 128)

In [5]:
# constants
START_TOKEN = '<START>'
END_TOKEN = '<END>'
NUM_SAMPLES = 10000
imdbDataset = load_dataset("stanfordnlp/imdb")

embeddingsFilepath = '/Users/josep/Desktop/Self/Learning/NLP/RNN/data/glove.6B.300d.txt'

# helper functions
def read_corpus(dataset):
    files = dataset["train"]["text"][:NUM_SAMPLES]
    return [[START_TOKEN] + [re.sub(r'[^\w]', '', w.lower()) for w in f.split(" ")] + [END_TOKEN] for f in files]


def embedding_for_vocab(filepath, words, dimensions):
    vocab_size = len(words)
    embeddings = np.zeros((vocab_size, dimensions))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in words.keys():
                index = words[word]
                embeddings[index] = np.array(vector)[:dimensions]
    return embeddings

imdbCorpus = read_corpus(imdbDataset)

corpusWords = [y for x in imdbCorpus for y in x]
corpusWords = list(set(corpusWords))
word2ind={}
for i in range(len(corpusWords)+1):
    word2ind[corpusWords[i-1]] = i
word2ind['<PAD>'] = 0
embeddings = embedding_for_vocab(embeddingsFilepath, word2ind, 300)

In [ ]:
test_transformer = bt.transformer(
    embeddings=embeddings, word2ind=word2ind
    , input_layer_shape=100, input_layer_activation='relu'
    , hidden_layer_shapes=[100,100,100], hidden_layer_activations=['relu', 'relu', 'relu']
    , output_layer_activation='softmax'
)

In [14]:
test_transformer.input_layer.activation

'relu'